In [3]:
# from PCL import *
import numpy as np
import glob
import random
from segment2d import *
import nibabel as nib
from torchsummary import summary
from ipywidgets import interact
import matplotlib.pyplot as plt
import albumentations as A
from torchvision import transforms
import os
import csv
import kornia as K

In [6]:
cfg.PREDICT.MODEL = "resnet50"
cfg.TRAIN.LOSS = "MSELoss"

In [7]:
for i in range(1, 6):
    os.makedirs(f"./weights_isbi_{cfg.PREDICT.MODEL}_{cfg.TRAIN.LOSS}/fold{i}/", exist_ok=True)


In [7]:
from PCL import *
from tqdm import tqdm
from timm.optim import Nadam
import torch
from tqdm import tqdm
import wandb
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau
import wandb
from torch.utils.data import DataLoader
from torchvision import models
from tqdm import tqdm
from torchvision.models import resnet50
from tqdm import tqdm
from torchvision.models import ResNet50_Weights
model = resnet50(weights=ResNet50_Weights.DEFAULT)
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! load pretrained model !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
model1 = resnet50(weights=ResNet50_Weights.DEFAULT)

model = ConvNextEncoder(in_dim = cfg.DATA.DIM_SIZE, num_class=cfg.DATA.NUM_CLASS)
checkpoint = torch.hub.load_state_dict_from_url(url=convnext_urls["convnext_tiny_1k"], map_location="cpu", check_hash=True)
model.load_state_dict(checkpoint["model"], strict=False)
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! load pretrained model !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
model1 = ConvNextEncoder(in_dim = cfg.DATA.DIM_SIZE, num_class=cfg.DATA.NUM_CLASS)

if cfg.TRAIN.LOAD_CKPT:
    #load checkpoint
    try:
        checkpoint = torch.load(cfg.DIRS.SAVE_CONVNEXT + "last.pt")
        model1.load_state_dict(checkpoint)
        print("load last model")
    except:
        print("no checkpoint found")


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! load pretrained model !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! load pretrained model !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
no checkpoint found


In [ ]:
model1.state_dict()

In [ ]:
learner = PCL_Resnet50(
    model1,
    image_size = 224,
    hidden_layer_pixel = 'layer4',  # leads to output of 8x8 feature map for pixel-level learning
    hidden_layer_instance = -2,     # leads to output for instance-level learning
    projection_size = 256,          # size of projection output, 256 was used in the paper
    projection_hidden_size = 2048,  # size of projection hidden dimension, paper used 2048
    moving_average_decay = 0.99,    # exponential moving average decay of target encoder
    ppm_num_layers = 1,             # number of layers for transform function in the pixel propagation module, 1 was optimal
    ppm_gamma = 2,                  # sharpness of the similarity in the pixel propagation module, already at optimal value of 2
    distance_thres = 0.7,           # ideal value is 0.7, as indicated in the paper, which makes the assumption of each feature map's pixel diagonal distance to be 1 (still unclear)
    similarity_temperature = 0.3,   # temperature for the cosine similarity for the pixel contrastive loss
    alpha = 1.,                     # weight of the pixel propagation loss (pixpro) vs pixel CL loss
    use_pixpro = True,              # do pixel pro instead of pixel contrast loss, defaults to pixpro, since it is the best one
    cutout_ratio_range = (0.6, 0.8) # a random ratio is selected from this range for the random cutout
)

In [ ]:
model1.state_dict()

In [ ]:
model = SkipNet(in_dim = cfg.DATA.DIM_SIZE, num_class=cfg.DATA.NUM_CLASS)
segmenter = Segmenter(model, cfg.DATA.CLASS_WEIGHT, cfg.DATA.NUM_CLASS, 
                                        cfg.OPT.LEARNING_RATE, cfg.OPT.FACTOR_LR, cfg.OPT.PATIENCE_LR)

for layer in list(segmenter.model.children())[:2]:
    for parameter in layer.parameters():
        parameter.requires_grad = False

In [11]:
import segmentation_models_pytorch as smp
model = smp.UnetPlusPlus(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,          
    # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,                      # model output channels (number of classes in your dataset)
    activation = "softmax2d"
)
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! load pretrained model !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
# model.encoder.load_state_dict(torch.load(cfg.DIRS.SAVE_RESNET50 + "best_model.pt"))

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! load pretrained model !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
model.encoder.load_state_dict(torch.load(cfg.DIRS.SAVE_RESNET50 + "best_model.pt"))
model.encoder.requires_grad_(False)

In [12]:
model

UnetPlusPlus(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential

In [13]:
summary(model, (3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
├─ResNetEncoder: 1-1                     [-1, 3, 224, 224]         --
|    └─Conv2d: 2-1                       [-1, 64, 112, 112]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 112, 112]        128
|    └─ReLU: 2-3                         [-1, 64, 112, 112]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 56, 56]          --
|    └─Sequential: 2-5                   [-1, 256, 56, 56]         --
|    |    └─Bottleneck: 3-1              [-1, 256, 56, 56]         75,008
|    |    └─Bottleneck: 3-2              [-1, 256, 56, 56]         70,400
|    |    └─Bottleneck: 3-3              [-1, 256, 56, 56]         70,400
|    └─Sequential: 2-6                   [-1, 512, 28, 28]         --
|    |    └─Bottleneck: 3-4              [-1, 512, 28, 28]         379,392
|    |    └─Bottleneck: 3-5              [-1, 512, 28, 28]         280,064
|    |    └─Bottleneck: 3-6              [-1, 512, 28, 28] 

Layer (type:depth-idx)                   Output Shape              Param #
├─ResNetEncoder: 1-1                     [-1, 3, 224, 224]         --
|    └─Conv2d: 2-1                       [-1, 64, 112, 112]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 112, 112]        128
|    └─ReLU: 2-3                         [-1, 64, 112, 112]        --
|    └─MaxPool2d: 2-4                    [-1, 64, 56, 56]          --
|    └─Sequential: 2-5                   [-1, 256, 56, 56]         --
|    |    └─Bottleneck: 3-1              [-1, 256, 56, 56]         75,008
|    |    └─Bottleneck: 3-2              [-1, 256, 56, 56]         70,400
|    |    └─Bottleneck: 3-3              [-1, 256, 56, 56]         70,400
|    └─Sequential: 2-6                   [-1, 512, 28, 28]         --
|    |    └─Bottleneck: 3-4              [-1, 512, 28, 28]         379,392
|    |    └─Bottleneck: 3-5              [-1, 512, 28, 28]         280,064
|    |    └─Bottleneck: 3-6              [-1, 512, 28, 28] 

In [ ]:
from torchvision import models
from tqdm import tqdm
import torch.functional as F
import timm
from torchvision.models import ResNet50_Weights, resnet50, efficientnet_b5, EfficientNet_B5_Weights
from torchsummary import summary
resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
efficientnet = efficientnet_b5(weights=EfficientNet_B5_Weights.DEFAULT)


In [ ]:
import segmentation_models_pytorch as smp
x = torch.randn(1, 3, 224, 224)
model = smp.Linknet(
    encoder_name="efficientnet-b5",
    encoder_weights="imagenet",
    in_channels=3, 
    classes=2,
    activation = "softmax2d",
    # decoder_attention_type = "scse"
    
)
from segmentation_models_pytorch.encoders import get_preprocessing_fn

preprocess_input = get_preprocessing_fn('resnet50', pretrained='imagenet')

In [ ]:
class EfficientNetEncoder(nn.Module):
    def __init__(self, in_dim, num_class, segment_model_encoder):
        super(EfficientNetEncoder, self).__init__()
        self.in_dim = in_dim
        self.num_class = num_class
        self.segment_model_encoder = segment_model_encoder
        self.avg_pooling = nn.AdaptiveAvgPool2d((1, 1))
    def forward(self, x):
        x = self.segment_model_encoder(x)
        x = self.avg_pooling(x)
        return x

In [ ]:
efficientnet = EfficientNetEncoder(in_dim = cfg.DATA.DIM_SIZE, num_class=cfg.DATA.NUM_CLASS, segment_model_encoder=model.encoder)

In [ ]:
model.encoder.load_state_dict(efficientnet.segment_model_encoder.state_dict())

In [ ]:
a = glob.glob("data/brats/brats_npz/*")

In [ ]:
x = np.load(a[0])["flair"]
y = np.load(a[1])["flair"]

In [ ]:
x = np.expand_dims(x.transpose(-1,0,1), axis=0)
y = np.expand_dims(y.transpose(-1,0,1), axis=0)
z = np.concatenate([x,y], axis=0)
images = torch.from_numpy(z).float()

In [ ]:
learner = PCL_Resnet50(
    model,
    image_size = 224,
    hidden_layer_pixel = 'layer4',  # leads to output of 8x8 feature map for pixel-level learning
    hidden_layer_instance = -2,     # leads to output for instance-level learning
    projection_size = 256,          # size of projection output, 256 was used in the paper
    projection_hidden_size = 2048,  # size of projection hidden dimension, paper used 2048
    moving_average_decay = 0.99,    # exponential moving average decay of target encoder
    ppm_num_layers = 1,             # number of layers for transform function in the pixel propagation module, 1 was optimal
    ppm_gamma = 2,                  # sharpness of the similarity in the pixel propagation module, already at optimal value of 2
    distance_thres = 0.7,           # ideal value is 0.7, as indicated in the paper, which makes the assumption of each feature map's pixel diagonal distance to be 1 (still unclear)
    similarity_temperature = 0.3,   # temperature for the cosine similarity for the pixel contrastive loss
    alpha = 1.,                      # weight of the pixel propagation loss (pixpro) vs pixel CL loss
    use_pixpro = True,               # do pixel pro instead of pixel contrast loss, defaults to pixpro, since it is the best one
    cutout_ratio_range = (0.6, 0.8)  # a random ratio is selected from this range for the random cutout
)

In [ ]:
# learner = PCL_ConvNext(
#     model,
#     image_size = 224,
#     hidden_layer_pixel = 'stages',  # leads to output of 8x8 feature map for pixel-level learning
#     hidden_layer_instance = "final_layer",     # leads to output for instance-level learning
#     projection_size = 256,          # size of projection output, 256 was used in the paper
#     projection_hidden_size = 2048,  # size of projection hidden dimension, paper used 2048
#     moving_average_decay = 0.99,    # exponential moving average decay of target encoder
#     ppm_num_layers = 1,             # number of layers for transform function in the pixel propagation module, 1 was optimal
#     ppm_gamma = 2,                  # sharpness of the similarity in the pixel propagation module, already at optimal value of 2
#     distance_thres = 0.7,           # ideal value is 0.7, as indicated in the paper, which makes the assumption of each feature map's pixel diagonal distance to be 1 (still unclear)
#     similarity_temperature = 0.3,   # temperature for the cosine similarity for the pixel contrastive loss
#     alpha = 1.,                      # weight of the pixel propagation loss (pixpro) vs pixel CL loss
#     use_pixpro = True,               # do pixel pro instead of pixel contrast loss, defaults to pixpro, since it is the best one
#     cutout_ratio_range = (0.6, 0.8)  # a random ratio is selected from this range for the random cutout
# ).cuda()
learner = learner.cuda()
opt = torch.optim.Adam(learner.parameters(), lr=1e-4)

def sample_batch_images():
    return torch.randn(10, 3, 224, 224)

images = sample_batch_images().cuda()

In [ ]:
images.shape

In [ ]:
for i in tqdm(range(100000)):
    loss = learner(images) # if positive pixel pairs is equal to zero, the loss is equal to the instance level loss

    opt.zero_grad()
    loss.backward()
    if i % 100 == 0:
        print(loss.item())
    opt.step()
    learner.update_moving_average() # update moving average of target encoder

# after much training, save the improved model for testing on downstream task
# torch.save(resnet, 'improved-resnet.pt')

In [ ]:
learner.state_dict()

In [ ]:
model.state_dict()

In [ ]:
model1.state_dict()